Задача - cоздать механизм исправления ошибок в словах пользовательского ввода

### Постановка задачи.

Если Х - исходное предложение, которое может содержать орфографические ошибки или опечатки, то алгоритм А должен произвести предложение Y, такое что:
1. Ошибки и опечатки исправлены.
2. Изначально корректные части предложения остались без изменения.
3. Смысл не поменялся.
4. Стиль не поменялся.


Также необходимо определиться с тем, что же такое опечатка/ошибка.

1. Вставка символа. (Пошел гулять -> ПошШел гулять)
2. Удаление символа. (Пошел гулять -> Пошл гулять)
3. Замена символа. (Пошел гулять -> Пошел гулИть)
4. Перестановка двух соседних символов. (Пошел гулять -> ПоЕШл гулять)
5. Вставка пробела. (Пошел гулять -> По шел гулять)
6. Удаление пробела. (Пошел гулять -> Пошелгулять)

Ошибку между строками будем определять с помощью расстояния Дамерау-Левенштейна - это мера разницы двух строк символов, определяемая как минимальное количество операций вставки, удаления, замены и транспозиции (перестановки двух соседних символов), необходимых для перевода одной строки в другую.

Вначале были собраны следующие датасеты:

1. RUSpellRU - данные собранные из "Живого Журнала"
2. MultidomainGold - примеры из 7 текстовых источников, включая открытую сеть, новости, социальные сети, обзоры, субтитры, политические документы и литературные произведения
3. MedSpellChecker - тексты с ошибками из медицинского анамнеза.
4. GitHubTypoCorpusRu - орфографические ошибки и опечатки в коммитах с GitHub;

Data Fields:
* sources (str): оригинальное предложение.
* corrections (str): исправленное предложение.
* domain (str): домен, из которого взято предложение.

В качестве решения будут использоваться seq2seq модели - это модель, принимающая на вход последовательность элементов (слов, букв, признаков изображения и т.д.) и возвращающая другую последовательность элементов.

Алгоритм
1. В декодере RNN получает эмбеддинг <END> токена и первоначальное скрытое состояние.
2. RNN обрабатывает входной элемент, генерирует выход и новый вектор скрытого состояния (h4). Выход отбрасывается.
3. Механизм внимания использует скрытые состояния энкодера и вектор h4 для вычисления контекстного вектора (C4) на данном временном отрезке.
4. Вектора h4 и C4 конкатенируются в один вектор.
5. Этот вектор пропускается через нейронную сеть прямого распространения (feedforward neural network, FFN), обучаемую совместно с моделью.
6. Вывод FFN сети указывает на выходное слово на данном временном отрезке.
7. Алгоритм повторяется для следующего временного отрезка.

Тестирование модели.

В качестве моделей для тестирования былы взяты две модели: RuM2M100-1.2B и RuM2M100-418M



In [51]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

path_to_model = "ai-forever/RuM2M100-1.2B" # Путь к модели на HuggingFace

model = M2M100ForConditionalGeneration.from_pretrained(path_to_model)
tokenizer = M2M100Tokenizer.from_pretrained(path_to_model, src_lang="ru", tgt_lang="ru")

sentence = "Я пришол в ДВФУ однако он был закрыт"

encodings = tokenizer(sentence, return_tensors="pt")
generated_tokens = model.generate(
        **encodings, forced_bos_token_id=tokenizer.get_lang_id("ru"))
answer = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(answer[0])



Я пришел в ДВФУ однако он был закрыт


In [35]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

sentence = "Я пришол в ДВФУ однако он был закрыт"

path_to_model = "ai-forever/RuM2M100-418M"

model = M2M100ForConditionalGeneration.from_pretrained(path_to_model)
tokenizer = M2M100Tokenizer.from_pretrained(path_to_model, src_lang="ru", tgt_lang="ru")


encodings = tokenizer(sentence, return_tensors="pt")
generated_tokens = model.generate(
        **encodings, forced_bos_token_id=tokenizer.get_lang_id("ru"))
answer = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(answer[0])



Я пришел в ДВФУ, однако он был закрыт.


In [6]:
model

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): Embedding(14341, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): Embedding(14341, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((

In [20]:
import pandas as pd

In [21]:
test_df = pd.read_csv("data/MultidomainGold.csv")

In [36]:
def test_model(df, model, encodings, print_bool=False): # Функция для тестирования функционала модели
    test_sentences_source = df['source']
    test_sentences_correction = df['correction']
    answer_from_model_list = []
    for sentence in test_sentences_source:
        encodings = tokenizer(sentence, return_tensors="pt")
        generated_tokens = model.generate(
                **encodings, forced_bos_token_id=tokenizer.get_lang_id("ru"))
        answer = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        answer_from_model_list.append(answer)
    if print_bool:
        for sentence, answer, correction in zip(test_sentences_source, answer_from_model_list, test_sentences_correction):
            print(f"Исходное предложение:     {sentence}")
            print(f"Исправленное предложение: {answer[0]}")
            print(f"Корректное предложение:   {correction}")
            print()
    
    return correction

In [49]:
# Тестирование RuM2M100-418M
path_to_model = "ai-forever/RuM2M100-418M"

model = M2M100ForConditionalGeneration.from_pretrained(path_to_model)
tokenizer = M2M100Tokenizer.from_pretrained(path_to_model, src_lang="ru", tgt_lang="ru")

In [50]:
correction = test_model(test_df[10:15], model, tokenizer, True),

Исходное предложение:     Сувенир с "инклюзом" Грибочки из Балтийского янтаря!
Исправленное предложение: Сувенир с "инклюзом" Грибочки из Балтийского янтаря.
Корректное предложение:   Сувенир с "инклюзом" Грибочки из Балтийского янтаря!

Исходное предложение:     Сама процедура получения ВНЖ начинаеться с регистрации ООО в Словении.
Исправленное предложение: Сама процедура получения ВНЖ начинается с регистрации ООО в Словении.
Корректное предложение:   Сама процедура получения ВНЖ начинаеться с регистрации ООО в Словении.

Исходное предложение:     В минувшие выхожные состоялось торжественное открытие 10-го, юбилейного сезона ГУМ-Катка.
Исправленное предложение: В минувшие выходные состоялось торжественное открытие 10-го юбилейного сезона ГУМ-карта.
Корректное предложение:   В минувшие выходные состоялось торжественное открытие 10-го, юбилейного сезона ГУМ-Катка.

Исходное предложение:     Прооессионал может всегда отличить "богему" от "участника", т.к. цены на их услуги сильно кусаютс

In [46]:
# Тестирование RuM2M100-1.2B
path_to_model = "ai-forever/RuM2M100-1.2B"

model = M2M100ForConditionalGeneration.from_pretrained(path_to_model)
tokenizer = M2M100Tokenizer.from_pretrained(path_to_model, src_lang="ru", tgt_lang="ru")

In [48]:
correction = test_model(test_df[10:15], model, tokenizer, True),

Исходное предложение:     Сувенир с "инклюзом" Грибочки из Балтийского янтаря!
Исправленное предложение: Сувенир с "инклюзом" Грибочки из Балтийского янтаря!
Корректное предложение:   Сувенир с "инклюзом" Грибочки из Балтийского янтаря!

Исходное предложение:     Сама процедура получения ВНЖ начинаеться с регистрации ООО в Словении.
Исправленное предложение: Сама процедура получения ВНЖ начинаеться с регистрации ООО в Словении.
Корректное предложение:   Сама процедура получения ВНЖ начинаеться с регистрации ООО в Словении.

Исходное предложение:     В минувшие выхожные состоялось торжественное открытие 10-го, юбилейного сезона ГУМ-Катка.
Исправленное предложение: В минувшие выходные состоялось торжественное открытие 10-го, юбилейного сезона ГУМ-Катка.
Корректное предложение:   В минувшие выходные состоялось торжественное открытие 10-го, юбилейного сезона ГУМ-Катка.

Исходное предложение:     Прооессионал может всегда отличить "богему" от "участника", т.к. цены на их услуги сильно кусаю

Вывод: Можем заметить, что обе модели работают неплохо и ошибки возникают только со специфичными и редкими словами.

RuM2M100-1.2B работает в два раза медленне RuM2M100-418M, поэтому я бы использовал вторую модель.

RuM2M100-1.2B из-за большего количество параметров способен разпознавать сложные опечатки, но я считаю, что подобные ошибки будут крайне редко, поэтому модели RuM2M100-418M должно быть достаточно.

В main.py буду использовать RuM2M100-418M